In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time
import os
import datetime as dt
import requests

option = webdriver.ChromeOptions()
option.add_argument("start-maximized")


driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=option)

In [2]:
user = input('Enter user name')
password = input('Enter Password')
fromDate = input('Enter from date in dd-mm-yyyy format')
toDate = input('Enter to date in dd-mm-yyyy format')

In [ ]:
url = 'https://reports.shoonya.com/Account/Login'
driver.get(url)
time.sleep(10)
usernameField = driver.find_element('name', 'UserName')
usernameField.send_keys(user)
passwordField = driver.find_element('name', 'Password')
passwordField.send_keys(password)
submitButton = driver.find_element('id', 'submit-btn')
submitButton.click()
cookies = driver.get_cookies()
aspxauth = cookies[0]['value']
aspsessionid = cookies[1]['value']
# close browser window
driver.quit()

In [ ]:
dataUrl = 'https://reports.shoonya.com/Home/BillSummaryDerivative'
payload = {
    'FromDate': fromDate,
    'ToDate': toDate
}
cookies = {
    "ASP.NET_SessionId": aspsessionid,
    ".ASPXAUTH": aspxauth
}
res = requests.post(dataUrl, cookies=cookies, data=payload)


In [ ]:
df = pd.read_html(res.text)[3]
df = df[['Trade Date', 'Bill Amount', 'Total Expenses']]
df

In [ ]:
df_mod = df.iloc[:-1, :]
df_mod = df_mod[df_mod['Bill Amount'] < 200000]

In [ ]:
df_mod['CUMSUM'] = df_mod['Bill Amount'].cumsum()


In [ ]:
df_mod.plot(x='Trade Date', y=['CUMSUM'], figsize=(20, 10), color='green')

In [ ]:
drawdowns = []
runningMax = 0

for date, val in zip(df_mod['Trade Date'], df_mod['CUMSUM']):
    date = (dt.datetime.strptime(date, '%d/%m/%Y')).strftime('%Y-%m-%d')
    if (val>=runningMax):
        drawdowns.append([date, 0])
        runningMax = val
    else:
        drawdowns.append([date, val-runningMax])

drawdownDf = pd.DataFrame(drawdowns)
drawdownDf.columns = ['Date', 'Drawdown']
drawdownDf.set_index('Date', inplace=True)
drawdownDf

In [ ]:
drawdownDf.plot(figsize=(20, 10), color='red')